In [2]:
import autograd.numpy as np
import matplotlib.pyplot as plt

from ilqr4i.functions import sample_trajectory
from ilqr4i.algorithms.kalman import KalmanSmoother
from ilqr4i.algorithms.lqr import iLQR
from ilqr4i.models.dynamics.linear import LinearDynamics
from ilqr4i.models.measurements.gaussian import LinearGaussianMeasurement
from ilqr4i.models.priors.gaussian import GaussianPrior
from ilqr4i.plotters import *

plt.style.use(["style.mplstyle"])

OSError: 'style.mplstyle' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

## General Parameters

In [ ]:
num_steps = 100

Nx = 1
Nu = 1
Ny = 1

: 

## Create the Generative Model
### Dynamics

In [ ]:
A = np.random.uniform(0.8, 1.0, size=(Nx, Nx))
B = np.random.uniform(-1.0, 1.0, size=(Nx, Nu))
dynamics = LinearDynamics(Nx, Nu, A, B)

: 

### Measurements

In [ ]:
C = np.random.uniform(-2.0, 2.0, size=(Ny, Nx))
y_cov = 0.5*np.eye(Ny)
meas_model = LinearGaussianMeasurement(Ny, y_cov, C)

: 

### Priors

In [ ]:
x0_prior = GaussianPrior(Nx, np.zeros(Nx), 2*np.eye(Nx))
u_prior = GaussianPrior(Nu, np.zeros(Nu), 0.5*np.eye(Nu))

: 

## Sample a Trajectory

In [ ]:
true_xs, true_us, ys = sample_trajectory(
    dynamics,
    meas_model,
    u_prior,
    x0_prior,
    num_steps
)

: 

## Run Inference Methods

In [ ]:
ilqr = iLQR(
    dynamics,
    meas_model,
    u_prior,
    x0_prior
)

us_init = [u_prior.sample(t) for t in range(num_steps)]

ilqr_xs, ilqr_us = ilqr(ys, us_init)

: 

In [ ]:
kalman_smoother = KalmanSmoother(
    dynamics,
    meas_model,
    u_prior,
    x0_prior
)

kalman_xs, kalman_us = kalman_smoother(ys)

: 

## Plot the Results

In [ ]:
fig, ax = plt.subplots(Nx, 1, figsize=(6, 0.5+Nx))

plot_groundtruths([ax], true_xs)
plot_gaussians([ax], ilqr_xs, "b", label="ilqr")
plot_gaussians([ax], kalman_xs, "r", label="kalman")
plt.show()

: 

In [ ]:
fig, ax = plt.subplots(Nu, 1, figsize=(6, 0.5+Nu))

plot_groundtruths([ax], true_us)
plot_gaussians([ax], ilqr_us, "b", label="ilqr")
plot_gaussians([ax], kalman_us, "r", label="kalman")
plt.show()

: 

In [ ]:
fig, ax = plt.subplots(Ny, 1, figsize=(6, 0.5+Ny))

plot_groundtruths([ax], ys, markers=True)
plot_measurements([ax], meas_model, ilqr_xs, "b", label="ilqr")
plot_measurements([ax], meas_model, kalman_xs, "r", label="kalman")
plt.show()

: 